In [77]:
from ber_metrics import BEREstimator
import numpy as np
import pickle
import pandas as pd

## Adult Income

In [78]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data.pkl', 'rb'))
X_train = X_train.drop(labels=['race_Other', 'race_White', 'sex_Male'], axis=1)
X = X_train[['capital-gain', 'capital-loss', 'hours-per-week', 'age', 'education-num']]
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(adult_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))


Bhattacharrya upper bound: 0.028454684268292584
Mahalanobis upper bound: 0.23843784291450348


In [5]:
# for the different groups
print("----Decoupled Race-----")
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['race_Black']==1], y_train[X_train['race_Black']==1]
X_test, y_test = X_test[X_test['race_Black']==1], y_test[X_test['race_Black']==1]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print(X_train.shape, y_train.shape)
print("Black")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))

X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['race_White']==1], y_train[X_train['race_White']==1]
X_test, y_test = X_test[X_test['race_White']==1], y_test[X_test['race_White']==1]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("White")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))

print("-----Decoupled Sex-------")
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['sex_Male']==1], y_train[X_train['sex_Male']==1]
X_test, y_test = X_test[X_test['sex_Male']==1], y_test[X_test['sex_Male']==1]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Male")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))

X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data_nonsingular.pkl', 'rb'))
X_train, y_train = X_train[X_train['sex_Male']==0], y_train[X_train['sex_Male']==0]
X_test, y_test = X_test[X_test['sex_Male']==0], y_test[X_test['sex_Male']==0]
X_train = X_train.drop(labels=['race_Asian-Pac-Islander', 'race_Black', 'race_Other', 'race_White', 'sex_Male'], axis=1)
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Male")
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound()))


----Decoupled Race-----
(2817, 91) (2817,)
Black
Mahalanobis upper bound: 0.14218138104033312
White
Mahalanobis upper bound: 0.2463563862753147
-----Decoupled Sex-------
Male
Mahalanobis upper bound: 0.28243467283922163
Male
Mahalanobis upper bound: 0.12619764677473774


## COMPAS Recidivism

In [3]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_violent_nonsingular.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 0.060966314167847147
Mahalanobis upper bound: 0.19586095536150863


In [4]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_violent_nonsingular.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 0.08397164606295981
Mahalanobis upper bound: 0.19586095536150863
Nearest Neighbor upper bound: 0.15114873035066506


In [5]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_arrest_nonsingular.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
# print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound()))

Bhattacharrya upper bound: 2.5016397436853592e-05
Mahalanobis upper bound: 0.4223692451173532


In [4]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_arrest_nonsingular.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
arrest_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(arrest_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(arrest_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(arrest_estimator.nn_bound()))

Bhattacharrya upper bound: 0.18714772071486613
Mahalanobis upper bound: 0.4223692451173532
Nearest Neighbor upper bound: 0.437726723095526
